In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/creditcardfraud/creditcard.csv


In [3]:
path='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillUp/labs/ML-FinalAssignment/Weather_Data.csv'
!wget -O Weather_Data.csv https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillUp/labs/ML-FinalAssignment/Weather_Data.csv


--2024-11-04 13:11:06--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillUp/labs/ML-FinalAssignment/Weather_Data.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 284201 (278K) [text/csv]
Saving to: 'Weather_Data.csv'

Weather_Data.csv    100%[===================>] 277.54K  1.80MB/s    in 0.2s    

2024-11-04 13:11:07 (1.80 MB/s) - 'Weather_Data.csv' saved [284201/284201]



In [4]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix, accuracy_score
import sklearn.metrics as metrics


In [5]:
df = pd.read_csv("Weather_Data.csv")
df.head()

,Date,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2/1/2008,19.5,22.4,15.6,6.2,0.0,W,41,S,SSW,...,92,84,1017.6,1017.4,8,8,20.7,20.9,Yes,Yes
1,2/2/2008,19.5,25.6,6.0,3.4,2.7,W,41,W,E,...,83,73,1017.9,1016.4,7,7,22.4,24.8,Yes,Yes
2,2/3/2008,21.6,24.5,6.6,2.4,0.1,W,41,ESE,ESE,...,88,86,1016.7,1015.6,7,8,23.5,23.0,Yes,Yes
3,2/4/2008,20.2,22.8,18.8,2.2,0.0,W,41,NNE,E,...,83,90,1014.2,1011.8,8,8,21.4,20.9,Yes,Yes
4,2/5/2008,19.7,25.7,77.4,4.8,0.0,W,41,NNE,W,...,88,74,1008.3,1004.8,8,8,22.5,25.5,Yes,Yes


In [6]:
df_sydney_processed = pd.get_dummies(data=df, columns=['RainToday', 'WindGustDir', 'WindDir9am', 'WindDir3pm'])

In [7]:
df_sydney_processed.replace(['No', 'Yes'], [0,1], inplace=True)

/tmp/ipykernel_30/288546165.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_sydney_processed.replace(['No', 'Yes'], [0,1], inplace=True)


In [9]:
df_sydney_processed = pd.get_dummies(data=df, columns=['RainToday', 'WindGustDir', 'WindDir9am', 'WindDir3pm'])

In [12]:
categorical_columns = df.select_dtypes(include=['object', 'category']).columns
print(categorical_columns)

Index(['Date', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday',
       'RainTomorrow'],
      dtype='object')


In [14]:
df_sydney_processed.drop('Date',axis=1,inplace=True)

KeyError: "['Date'] not found in axis"

In [16]:
df_sydney_processed.replace(['No', 'Yes'], [0,1], inplace=True)
df_sydney_processed = df_sydney_processed.astype(float)

In [17]:
features = df_sydney_processed.drop(columns='RainTomorrow', axis=1)
Y = df_sydney_processed['RainTomorrow']

In [18]:
print(df_sydney_processed.columns.tolist())
print(Y)

['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine', 'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am', 'Temp3pm', 'RainTomorrow', 'RainToday_No', 'RainToday_Yes', 'WindGustDir_E', 'WindGustDir_ENE', 'WindGustDir_ESE', 'WindGustDir_N', 'WindGustDir_NE', 'WindGustDir_NNE', 'WindGustDir_NNW', 'WindGustDir_NW', 'WindGustDir_S', 'WindGustDir_SE', 'WindGustDir_SSE', 'WindGustDir_SSW', 'WindGustDir_SW', 'WindGustDir_W', 'WindGustDir_WNW', 'WindGustDir_WSW', 'WindDir9am_E', 'WindDir9am_ENE', 'WindDir9am_ESE', 'WindDir9am_N', 'WindDir9am_NE', 'WindDir9am_NNE', 'WindDir9am_NNW', 'WindDir9am_NW', 'WindDir9am_S', 'WindDir9am_SE', 'WindDir9am_SSE', 'WindDir9am_SSW', 'WindDir9am_SW', 'WindDir9am_W', 'WindDir9am_WNW', 'WindDir9am_WSW', 'WindDir3pm_E', 'WindDir3pm_ENE', 'WindDir3pm_ESE', 'WindDir3pm_N', 'WindDir3pm_NE', 'WindDir3pm_NNE', 'WindDir3pm_NNW', 'WindDir3pm_NW', 'WindDir3pm_S', 'WindDir3pm_

In [20]:
x_train, x_test, y_train, y_test = train_test_split(df_sydney_processed, Y, test_size=0.2, random_state=10)       
print('x_train.shape=', x_train.shape, 'Y_train.shape=', y_train.shape)
print('X_test.shape=', x_test.shape, 'Y_test.shape=', y_test.shape)

x_train.shape= (2616, 67) Y_train.shape= (2616,)
X_test.shape= (655, 67) Y_test.shape= (655,)


# linearRegression

In [21]:
LinearReg =LinearRegression()
LinearReg.fit(x_train, y_train)

LinearRegression()

In [22]:
predictions = LinearReg.predict(x_test)
predictions

array([ 2.93362298e-15,  7.61566517e-19,  1.00000000e+00,  1.00000000e+00,
        1.61145346e-15,  1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
        1.00000000e+00,  1.05162432e-16, -8.12646431e-16,  1.00000000e+00,
       -1.58422768e-15,  4.66434583e-15, -1.84673085e-15,  1.00000000e+00,
       -3.80920820e-16, -2.62718197e-15,  9.56398219e-16, -3.01830467e-15,
        8.19700818e-16,  1.00000000e+00,  1.94964864e-15, -2.65952756e-15,
        1.72514173e-15, -4.75802586e-16,  1.00000000e+00,  9.23780459e-16,
        1.00000000e+00,  7.77688913e-15,  1.00000000e+00, -1.72027038e-15,
       -4.14407037e-16, -2.14984018e-15,  3.57498497e-15,  1.00000000e+00,
       -8.95367463e-17,  1.00000000e+00, -1.64855145e-15,  5.46170369e-16,
       -3.06087144e-15,  1.00000000e+00,  1.00000000e+00, -3.08300604e-15,
        6.15803983e-16,  1.00000000e+00,  6.31794944e-16,  3.33596669e-15,
        1.00000000e+00,  1.00000000e+00,  4.59246808e-15, -2.26111282e-15,
       -1.08694562e-15, -

In [23]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [24]:
LinearRegression_MAE = mean_absolute_error(y_test, predictions)
LinearRegression_MSE = mean_squared_error(y_test, predictions)
LinearRegression_R2 = r2_score(y_test, predictions)
Report = pd.DataFrame({
    'Metric': ['Mean Absolute Error (MAE)', 'Mean Squared Error (MSE)', 'R-squared (R²)'],
    'Value': [LinearRegression_MAE, LinearRegression_MSE, LinearRegression_R2]
})
print(Report)

                      Metric         Value
0  Mean Absolute Error (MAE)  1.694893e-15
1   Mean Squared Error (MSE)  5.259643e-30
2             R-squared (R²)  1.000000e+00


# KNN

In [25]:
KNN = KNeighborsClassifier()
KNN.fit(x_train, y_train)

KNeighborsClassifier()

In [26]:
predictions = KNN.predict(x_test)

In [27]:
KNN_Accuracy_Score = accuracy_score(y_test, predictions)
KNN_JaccardIndex = jaccard_score(y_test, predictions)
KNN_F1_Score =  f1_score(y_test, predictions, average='weighted')  
Report = pd.DataFrame({
    'Metric': ['KNN_Accuracy_Score', 'KNN_JaccardIndex', 'KNN_F1_Score'],
    'Value': [KNN_Accuracy_Score, KNN_JaccardIndex, KNN_F1_Score]
})

In [28]:
print(Report)

               Metric     Value
0  KNN_Accuracy_Score  0.825954
1    KNN_JaccardIndex  0.479452
2        KNN_F1_Score  0.818020


# Decision tree

In [29]:
tree=DecisionTreeClassifier()
tree.fit(x_train, y_train)

DecisionTreeClassifier()

In [30]:
predictions=tree.predict(x_test)

In [32]:
Tree_Accuracy_Score =  accuracy_score(y_test, predictions)
Tree_JaccardIndex = jaccard_score(y_test, predictions)
Tree_F1_Score = f1_score(y_test, predictions, average='weighted')  
Report = pd.DataFrame({
    'Metric': ['KNN_Accuracy_Score', 'KNN_JaccardIndex', 'KNN_F1_Score'],
    'Value': [KNN_Accuracy_Score, KNN_JaccardIndex, KNN_F1_Score]
})
print(Report)

               Metric     Value
0  KNN_Accuracy_Score  0.825954
1    KNN_JaccardIndex  0.479452
2        KNN_F1_Score  0.818020


# logistic regression

In [40]:

logreg=LogisticRegression(random_state=1)
logreg.fit(x_train, y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=1)

In [42]:
predictions=logreg.predict(x_test)

In [43]:
Accuracy_Score =  accuracy_score(y_test, predictions)
JaccardIndex = jaccard_score(y_test, predictions)
F1_Score = f1_score(y_test, predictions, average='weighted')  
Report = pd.DataFrame({
    'Metric': ['Accuracy_Score', 'JaccardIndex', 'F1_Score'],
    'Value': [Accuracy_Score, JaccardIndex, F1_Score]
})
print(Report)

           Metric     Value
0  Accuracy_Score  0.893130
1    JaccardIndex  0.666667
2        F1_Score  0.891384


support vector machine

In [47]:

from sklearn.svm import LinearSVC


sklearn_svm = LinearSVC(class_weight='balanced', random_state=31, loss="hinge", fit_intercept=False)

sklearn_svm.fit(x_train, y_train)


/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(class_weight='balanced', fit_intercept=False, loss='hinge',
          random_state=31)

In [52]:

sklearn_pred = sklearn_svm.predict(x_test)
Accuracy_Score =  accuracy_score(y_test, sklearn_pred)
JaccardIndex = jaccard_score(y_test, sklearn_pred)
F1_Score = f1_score(y_test, sklearn_pred, average='weighted')  
Report = pd.DataFrame({
    'Metric': ['Accuracy_Score', 'JaccardIndex', 'F1_Score'],
    'Value': [Accuracy_Score, JaccardIndex, F1_Score]
})
print(Report)

           Metric     Value
0  Accuracy_Score  0.758779
1    JaccardIndex  0.141304
2        F1_Score  0.685358
